In [59]:
def unfold(record, cdgs, multiplier):
    cdgs *= multiplier
    record += "?"
    record *= multiplier
    record = tuple(record[:-1])
    return record, cdgs


def calculate_cdgs(record):
    cdgs = []
    cdg = 0
    for c in record:
        if c == "#":
            cdg += 1
        elif cdg:
            cdgs.append(cdg)
            cdg = 0
    if cdg:
        cdgs.append(cdg)
    return tuple(cdgs)


def arrays_partially_correct(full, partial):
    els = len(partial)
    if not els:
        return True
    if els > len(full) or partial[-1] > full[els - 1]:
        return False
    return partial[:-1] == full[: els - 1]


def solve_one(record, cdgs, tmp_record):
    n_correct = 0
    if len(tmp_record) == len(record):
        return calculate_cdgs(tmp_record) == cdgs
    c = record[len(tmp_record)]
    if c == "?":
        for nc in [".", "#"]:
            next_tmp_record = tmp_record + (nc,)
            next_tmp_cdgs = calculate_cdgs(next_tmp_record)
            if arrays_partially_correct(cdgs, next_tmp_cdgs):
                n_correct += solve_one(record, cdgs, next_tmp_record)
    else:
        next_tmp_record = tmp_record + (c,)
        n_correct += solve_one(record, cdgs, next_tmp_record)
    return n_correct


def solve_all(lines, multiplier):
    answers = []
    for line in lines:
        record, cdgs = line.split()
        cdgs = tuple(int(cdg) for cdg in cdgs.split(","))
        record, cdgs = unfold(record, cdgs, multiplier)
        n_ways = solve_one(record, cdgs, ())
        answers.append(n_ways)
    return answers

my_correct = 0
def solve(filename, multiplier=1):
    global my_correct
    with open(filename) as f:
        lines = f.readlines()
        lines = lines[:10]
    answers = solve_all(lines, 1)
    if multiplier == 1:
        print(sum(answers))
    else:
        answers2 = solve_all(lines, 2)
        final_answer = 0
        for i, (ans1, ans2) in enumerate(zip(answers, answers2)):
            if ans1 == ans2:
                final_answer += ans1
                # print(ans1)
            else:
                if ans2 % ans1 == 0:
                    factor = ans2 // ans1
                    final_multiplier = factor ** (multiplier - 1)
                    ans = ans1 * final_multiplier
                else:
                    record, cdgs = lines[i].split()
                    cdgs = tuple(int(cdg) for cdg in cdgs.split(","))
                    record, cdgs = unfold(record, cdgs, multiplier)
                    ans = solve_one(record, cdgs, ())
                final_answer += ans
                # print('is_correct', ans == correct_2[i])
                # print('my answer', ans)
                # print('correct', correct_2[i])
                # print('ans1', ans1)
                # print('ans2', ans2)
                # print('factor', ans2 / ans1)
                # print('final_multiplier', final_multiplier)
                # print('-' * 10)

                my_correct += ans == correct_2[i]
        # print(final_answer)


# print("Part one:")
# solve("test_input.txt")
# solve("input.txt")
# print("Part two:")
# solve("test_input.txt", 5)
solve("input.txt", 5)

print('my_correct', my_correct)

KeyboardInterrupt: 

39815426811630 wrong

65607131946466 right

In [63]:
from functools import cache


# @cache
def num_valid_solutions(record: str, groups: tuple[int, ...]) -> int:
    if not record:
        # if there are no more spots to check;
        # our only chance at success is if there are no `groups` left
        return len(groups) == 0

    if not groups:
        # if there are no more groups the only possibility of success is that there are no `#` remaining
        # here, `?` are treated as `.`, so no recursion is necessary
        return "#" not in record

    char, rest_of_record = record[0], record[1:]

    if char == ".":
        # dots are ignores, so keep recursing
        return num_valid_solutions(rest_of_record, groups)

    if char == "#":
        group = groups[0]
        # we're at the start of a group! make sure there are enough here to fill the first group
        # to be valid, we have to be:
        if (
            # long enough to match
            len(record) >= group
            # made of only things that can be `#` (no `.`)
            and all(c != "." for c in record[:group])
            # either at the end of the record (allowed)
            # or the next character isn't also a `#` (would be too big)
            and (len(record) == group or record[group] != "#")
        ):
            return num_valid_solutions(record[group + 1 :], groups[1:])

        return 0

    if char == "?":
        return num_valid_solutions(f"#{rest_of_record}", groups) + num_valid_solutions(
            f".{rest_of_record}", groups
        )

    raise ValueError(f"unknown char: {char}")

correct_2 = []

def solve_line(line: str, with_multiplier=False) -> int:
    record, raw_shape = line.split()
    shape = tuple(map(int, raw_shape.split(",")))

    if with_multiplier:
        record = "?".join([record] * 5)
        shape *= 5

    sols = num_valid_solutions(record, shape)
    correct_2.append(sols)
    return sols


def part_1(inputf) -> int:
    return sum(solve_line(line) for line in inputf)

def part_2(inputf) -> int:
    return sum(solve_line(line, with_multiplier=True) for line in inputf)


lines = open("input.txt").readlines()
# print("Part one:")
# p1 = part_1(lines)
# print(">", p1)
# print("Part two:")
p2 = part_2(lines)
print(">", p2)

In [17]:
from functools import cache

""" Problem 1
In the giant field just outside, the springs are arranged into rows. For each row, the condition records show every spring and whether it is operational (.) or damaged (#). This is the part of the condition records that is itself damaged; for some springs, it is simply unknown (?) whether the spring is operational or damaged.

However, the engineer that produced the condition records also duplicated some of this information in a different format! After the list of springs for a given row, the size of each contiguous group of damaged springs is listed in the order those groups appear in the row. This list always accounts for every damaged spring, and each number is the entire size of its contiguous group (that is, groups are always separated by at least one operational spring: #### would always be 4, never 2,2).

So, condition records with no unknown spring conditions might look like this:

#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1
However, the condition records are partially damaged; some of the springs' conditions are actually unknown (?). For example:

???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
Equipped with this information, it is your job to figure out how many different arrangements of operational and broken springs fit the given criteria in each row.

In the first line (???.### 1,1,3), there is exactly one way separate groups of one, one, and three broken springs (in that order) can appear in that row: the first three unknown springs must be broken, then operational, then broken (#.#), making the whole row #.#.###.

The second line is more interesting: .??..??...?##. 1,1,3 could be a total of four different arrangements. The last ? must always be broken (to satisfy the final contiguous group of three broken springs), and each ?? must hide exactly one of the two broken springs. (Neither ?? could be both broken springs or they would form a single contiguous group of two; if that were true, the numbers afterward would have been 2,3 instead.) Since each ?? can either be #. or .#, there are four possible arrangements of springs.

The last line is actually consistent with ten different arrangements! Because the first number is 3, the first and second ? must both be . (if either were #, the first number would have to be 4 or higher). However, the remaining run of unknown spring conditions have many different ways they could hold groups of two and one broken springs:

?###???????? 3,2,1
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
In this example, the number of possible arrangements for each row is:

???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 4 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 1 arrangement
????.######..#####. 1,6,5 - 4 arrangements
?###???????? 3,2,1 - 10 arrangements
Adding all of the possible arrangement counts together produces a total of 21 arrangements.

For each row, count all of the different arrangements of operational and broken springs that meet the given criteria. What is the sum of those counts?
"""

with open("input.txt", "r") as file:
    input = file.read().splitlines()


@cache
def count_arrangements(conditions, rules):
    if not rules:
        return 0 if "#" in conditions else 1
    if not conditions:
        return 1 if not rules else 0

    result = 0

    if conditions[0] in ".?":
        result += count_arrangements(conditions[1:], rules)
    if conditions[0] in "#?":
        if (
            rules[0] <= len(conditions)
            and "." not in conditions[: rules[0]]
            and (rules[0] == len(conditions) or conditions[rules[0]] != "#")
        ):
            result += count_arrangements(conditions[rules[0] + 1 :], rules[1:])

    return result


solution1 = 0
solution2 = 0
for line in input:
    conditions, rules = line.split()
    rules = eval(rules)
    solution1 += count_arrangements(conditions, rules)

    conditions = "?".join([conditions] * 5)
    rules = rules * 5
    solution2 += count_arrangements(conditions, rules)

print("Solution 1:", solution1)
print("Solution 2:", solution2)

Solution 1: 7490
Solution 2: 65607131946466


In [56]:
1*4*4*4*4

256